## Import Packages and Read Data

In [2]:
#import packages
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import *

/Users/kevinzhao/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/kevinzhao/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/kevinzhao/anaconda3/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be rem

In [3]:
# Read Data

filepath = '/Users/kevinzhao/Kaggle/Recruit_Restaurant/inputs/'

# main data
air_visit= pd.read_csv(filepath + 'air_visit_data.csv')

#reservation info
air_reserve = pd.read_csv(filepath + 'air_reserve.csv')
hpg_reserve = pd.read_csv(filepath + 'hpg_reserve.csv')

#store info
air_store_info = pd.read_csv(filepath + 'air_store_info.csv')
hpg_store_info = pd.read_csv(filepath + 'hpg_store_info.csv')

#japanese holidays
date_info = pd.read_csv(filepath + 'date_info.csv')

#ids to link stores
store_id_relation = pd.read_csv(filepath + 'store_id_relation.csv')

## Exploratory Data Analysis

#### Main data set: air_vist

In [92]:
air_visit.head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


In [93]:
# convert visit date to pandas datetime
#air_visit.visit_date = pd.to_datetime(air_visit['visit_date'])

In [94]:
# Train Data is from 1/1/2016 - 4/22/2017
air_visit.visit_date.min(), air_visit.visit_date.max()

('2016-01-01', '2017-04-22')

In [95]:
# 829 stores 
air_visit.air_store_id.nunique()

829

#### Store Info

In [96]:
air_store_info.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [97]:
hpg_store_info.head()

,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221


#### Reservation Info

In [98]:
air_reserve.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


In [99]:
hpg_reserve.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


#### Japanese Holidays

In [100]:
# 1 indicates holiday
date_info.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [101]:
# 35 of 517 days are holidays
date_info.holiday_flg.sum(), date_info.calendar_date.count()

(35, 517)

## Data Wrangling and Feature Engineering

In [4]:
# Rename columns to avoid variable overlap
air_store_info.columns = ['air_store_id','air_genre_name','air_area_name', 'air_latitude', 'air_longitude']
hpg_store_info.columns = ['hpg_store_id','hpg_genre_name','hpg_area_name', 'hpg_latitude', 'hpg_longitude']
air_reserve.columns = ['air_store_id', 'air_visit_datetime', 'air_reserve_datetime', 'air_reserve_visitors']
hpg_reserve.columns = ['hpg_store_id', 'hpg_visit_datetime', 'hpg_reserve_datetime', 'hpg_reserve_visitors']

### Joins

#### 1. Add Air store info to main data set

In [5]:
main_data = air_visit.merge(air_store_info, on='air_store_id', how='left')
main_data.head()

,air_store_id,visit_date,visitors,air_genre_name,air_area_name,air_latitude,air_longitude
0,air_ba937bf13d40fb24,2016-01-13,25,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
1,air_ba937bf13d40fb24,2016-01-14,32,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
2,air_ba937bf13d40fb24,2016-01-15,29,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
3,air_ba937bf13d40fb24,2016-01-16,22,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
4,air_ba937bf13d40fb24,2016-01-18,6,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


#### Add holiday and day of week

In [6]:
date_info.columns = ['visit_date','day_of_week','holiday_flg']

In [7]:
date_info

,visit_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0
5,2016-01-06,Wednesday,0
6,2016-01-07,Thursday,0
7,2016-01-08,Friday,0
8,2016-01-09,Saturday,0
9,2016-01-10,Sunday,0


In [8]:
main_data = main_data.merge(date_info, on = "visit_date", how='left')

In [9]:
main_data.head()

,air_store_id,visit_date,visitors,air_genre_name,air_area_name,air_latitude,air_longitude,day_of_week,holiday_flg
0,air_ba937bf13d40fb24,2016-01-13,25,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Wednesday,0
1,air_ba937bf13d40fb24,2016-01-14,32,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Thursday,0
2,air_ba937bf13d40fb24,2016-01-15,29,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Friday,0
3,air_ba937bf13d40fb24,2016-01-16,22,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Saturday,0
4,air_ba937bf13d40fb24,2016-01-18,6,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Monday,0


#### 2. Add Hpg store info to main data set

In [12]:
hpg_data = hpg_store_info.merge(store_id_relation, on ='hpg_store_id', how='left')
main_data_with_hpg = main_data.merge(hpg_data, on = 'air_store_id', how='outer')

#### 3. Pickle dataframe

In [14]:
pickle.dump(main_data_with_hpg, open('main_data_with_hpg.p', 'wb'))
main_data_with_hpg = pickle.load(open('main_data_with_hpg.p', 'rb'))

## Modeling

#### 1. Just for kicks lets start with linear regression

In [116]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(main_data[main_data.columns['air_latitude', 'air_longitude']],
                                                    main_data.visitors, random_state=42)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [109]:
X_train

,air_area_name,air_genre_name,air_latitude,air_longitude,air_store_id,visit_date,visitors
66672,Fukuoka-ken Fukuoka-shi Daimyō,Dining bar,33.589216,130.392813,air_fe22ef5a9cbef123,2016-09-12,27
179876,Hyōgo-ken Kōbe-shi Sumiyoshi Higashimachi,Cafe/Sweets,34.720228,135.265455,air_c28983412a7eefcf,2016-10-23,72
23787,Hyōgo-ken Kakogawa-shi Kakogawachō Kitazaike,Izakaya,34.756950,134.841177,air_e55abd740f93ecc4,2016-11-19,66
54269,Tōkyō-to Suginami-ku Asagayaminami,Other,35.699566,139.636438,air_efc80d3f96b3aff7,2016-09-28,8
62466,Tōkyō-to Minato-ku Shibakōen,Bar/Cocktail,35.658068,139.751599,air_c8265ecc116f2284,2017-03-11,3
160693,Hyōgo-ken Kōbe-shi Kumoidōri,Okonomiyaki/Monja/Teppanyaki,34.695124,135.197852,air_f927b2da69a82341,2016-10-13,4
240455,Tōkyō-to Suginami-ku Asagayaminami,Cafe/Sweets,35.699566,139.636438,air_5d945ade487cdf4d,2016-05-01,24
44672,Tōkyō-to Minato-ku Shibakōen,Dining bar,35.658068,139.751599,air_88c8e34baa79217b,2016-09-17,33
23996,Hyōgo-ken Kakogawa-shi Kakogawachō Kitazaike,Izakaya,34.756950,134.841177,air_3e6cea17a9d2c0f1,2016-03-22,13
220345,Tōkyō-to Shibuya-ku Shibuya,Okonomiyaki/Monja/Teppanyaki,35.661777,139.704051,air_8d61f49aa0373492,2017-02-13,42


In [107]:
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)

ValueError: could not convert string to float: '2017-01-09'